Pima Indians Diabetes(피마 인디언 당뇨병)

In [15]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../../Datasat/T2-2/diabetes.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Outcome')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 9), (154, 9), (614, 2), (154, 2))

2. EDA

In [16]:
print(X_train.head())
print(X_test.head())
print(y_train.head())

      id  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
147  147            2      106             64             35      119  30.5   
344  344            8       95             72              0        0  36.8   
390  390            1      100             66             29      196  32.0   
150  150            1      136             74             50      204  37.4   
132  132            3      170             64             37      225  34.5   

     DiabetesPedigreeFunction  Age  
147                     1.400   34  
344                     0.485   57  
390                     0.444   42  
150                     0.399   24  
132                     0.356   30  
      id  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
258  258            1      193             50             16      375  25.9   
220  220            0      177             60             29      478  34.6   
438  438            1       97             70             15      

In [17]:
print(X_train.info())
print(X_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, 147 to 116
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        614 non-null    int64  
 1   Pregnancies               614 non-null    int64  
 2   Glucose                   614 non-null    int64  
 3   BloodPressure             614 non-null    int64  
 4   SkinThickness             614 non-null    int64  
 5   Insulin                   614 non-null    int64  
 6   BMI                       614 non-null    float64
 7   DiabetesPedigreeFunction  614 non-null    float64
 8   Age                       614 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 48.0 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 154 entries, 258 to 174
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                    

In [18]:
print(X_train.describe())
print(X_test.describe())



               id  Pregnancies     Glucose  BloodPressure  SkinThickness  \
count  614.000000   614.000000  614.000000     614.000000     614.000000   
mean   386.978827     3.845277  120.705212      68.530945      20.982085   
std    219.881344     3.388952   32.047977      19.761915      15.965031   
min      0.000000     0.000000    0.000000       0.000000       0.000000   
25%    196.500000     1.000000   99.000000      62.000000       0.000000   
50%    388.000000     3.000000  117.000000      72.000000      23.000000   
75%    575.750000     6.000000  140.000000      80.000000      33.000000   
max    766.000000    17.000000  199.000000     114.000000      99.000000   

          Insulin         BMI  DiabetesPedigreeFunction         Age  
count  614.000000  614.000000                614.000000  614.000000  
mean    82.229642   31.948534                  0.477471   33.319218  
std    116.525772    7.734385                  0.339811   11.940953  
min      0.000000    0.000000      

3. 데이터 전처리, 분리

In [19]:
#train
print('Glucose', len(X_train[X_train['Glucose']==0]))
print('BloodPressure', len(X_train[X_train['BloodPressure']==0]))
print('SkinThickness', len(X_train[X_train['SkinThickness']==0]))
print('Insulin', len(X_train[X_train['Insulin']==0]))
print('BMI', len(X_train[X_train['BMI']==0]))


Glucose 5
BloodPressure 31
SkinThickness 175
Insulin 287
BMI 9


In [20]:
#test
print('Glucose', len(X_test[X_test['Glucose']==0]))
print('BloodPressure', len(X_test[X_test['BloodPressure']==0]))
print('SkinThickness', len(X_test[X_test['SkinThickness']==0]))
print('Insulin', len(X_test[X_test['Insulin']==0]))
print('BMI', len(X_test[X_test['BMI']==0]))

Glucose 0
BloodPressure 4
SkinThickness 52
Insulin 87
BMI 2


In [21]:
#포도당 이상치 삭제
del_idx = X_train[X_train['Glucose']==0].index
del_idx

Index([182, 75, 342, 502, 349], dtype='int64')

In [22]:
print('Glucose 이상치 삭제 전: ', X_train.shape, y_train.shape)
X_train = X_train.drop(index=del_idx, axis=1)
y_train = y_train.drop(index=del_idx, axis=1)
print('Glucose 이상치 삭제 후 : ', X_train.shape, y_train.shape)


Glucose 이상치 삭제 전:  (614, 9) (614, 2)
Glucose 이상치 삭제 후 :  (609, 9) (609, 2)


In [23]:
#포도당을 제외한 이상치를 평균값으로 대체
cols = ['BloodPressure', 'SkinThickness', 'Insulin','BMI']
cols_mean = X_train[cols].mean()
X_train.loc[:, cols].replace(0,cols_mean)      #전체 행에서 칼럼이 cols인것을 가져와
# 또는 X_train[cols] = X_train[cols].replace(0, cols_mean)

,BloodPressure,SkinThickness,Insulin,BMI
147,64.0,35.00000,119.000000,30.5
344,72.0,20.91133,82.866995,36.8
390,66.0,29.00000,196.000000,32.0
150,74.0,50.00000,204.000000,37.4
132,64.0,37.00000,225.000000,34.5
...,...,...,...,...
621,76.0,20.00000,82.866995,24.2
128,88.0,24.00000,145.000000,34.5
57,88.0,60.00000,110.000000,46.8
341,74.0,21.00000,73.000000,25.9


In [24]:
#스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.fit_transform(X_test[cols])


In [25]:
X = X_train.drop('id', axis=1)
test = X_test.drop('id',axis=1)

In [26]:
from sklearn.svm import SVC
model = SVC(random_state=2023)
model.fit(X, y_train['Outcome'])
pred = model.predict(test)

In [27]:
#오버피팅 되었을 때 점수가 잘 나올 수 있음(객관적 평가X, 벨리데이션 데이터로 평가 필요함)
round(model.score(X, y_train['Outcome']) * 100, 2)

83.42

In [28]:
output = pd.DataFrame({'idx': X_test.index, 'Outcome': pred})
output.head()

,idx,Outcome
0,258,1
1,220,1
2,438,0
3,130,1
4,730,0


In [ ]:
# 수험번호.csv로 출력
# output.to_csv('1234567.csv', index=False)